Trabajo elaborado grupalmente por:

Estudiante 1: antony.mendoza@upch.pe

Estudiante 2: JOHN.RODRIGUES@UPCH.PE

Estudiante 3: LEILY.LLANOS@UPCH.PE

# **Problema 1: Simulación de un ataque de reconocimiento y mitigación**

**Contexto**: Una empresa observa actividad sospechosa en sus servidores y sospecha de un
ataque de reconocimiento. Los estudiantes deben utilizar herramientas como dig,
traceroute, y whois para identificar la fuente y naturaleza del ataque, seguido por la
implementación de estrategias para mitigar y prevenir futuros ataques.

## **Paso 1: Identificación del ataque utilizando Python**

In [ ]:
!sudo apt-get update   #esto es necesario instalar para poder ejecutar la siguiente celda
!sudo apt-get install dnsutils


El propósito de obtener la dirección IP asociada con un dominio y luego realizar un traceroute a esa dirección IP para identificar la posible fuente de un ataque.

In [ ]:
import subprocess
import re
def run_dig(domain):
  result = subprocess.run(['dig', '+short', domain], stdout=subprocess.PIPE)
  return result.stdout.decode('utf-8').strip()

def run_traceroute(ip_address):
  result = subprocess.run(['traceroute', ip_address], stdout=subprocess.PIPE)
  return result.stdout.decode('utf-8')

def get_attack_source(trace_result):
    # Encuentra la dirección IP de origen en la salida del traceroute
    match = re.search(r'\((\d+\.\d+\.\d+\.\d+)\)', trace_result)
    if match:
        return match.group(1)
    else:
        return None

# imprimirá la dirección IP asociada con el dominio (si se encuentra),
# la salida del traceroute y la posible fuente de ataque
# (si se puede extraer una dirección IP de la salida del traceroute)
domain = 'suspicious.example.com'
ip_address = run_dig(domain)
if ip_address:
    print(f"IP Address for {domain} is {ip_address}")
    trace_result = run_traceroute(ip_address)
    print("Traceroute result:", trace_result)
    attack_source = get_attack_source(trace_result)
    if attack_source:
        print("Potential attack source:", attack_source)
    else:
        print("No attack source found in traceroute result")

## **Paso 2: Análisis de tráfico**

In [ ]:
!tcpdump -i eth0 -c 100 -w capture.pcap #esto ayuda a capturar el tráfico de red de una interfaz.
                                        #lo cuál se guarda en un archivo pcap para usarlo en la siguiente celda

In [ ]:
import os
def capture_packets(interface='eth0', count=10, outfile='capture.pcap'):
  cmd = f"tcpdump -i {interface} -c {count} -w {outfile}"
  os.system(cmd)
def analyze_packets(pcap_file):
  try:
    from scapy.all import rdpcap, IP
    packets = rdpcap(pcap_file)
    for packet in packets:
      if packet.haslayer(IP):
        print(f"Source: {packet[IP].src}, Destination: {packet[IP].dst}")
  except ImportError:
    print("Scapy is not installed. Install it to analyze packets.")
# Ejemplo de uso
capture_packets()
analyze_packets('capture.pcap')

## **Paso 3: Mitigación y prevención**

In [ ]:
def block_ip(ip_address):
  cmd = f"iptables -A INPUT -s {ip_address} -j DROP"
  os.system(cmd)
  print(f"Blocked IP address {ip_address}")

# Bloquear la dirección IP encontrada en el Paso 1
if ip_address:
  block_ip(ip_address)

Ejemplo de implementación de la solución sobre este problema en base al contexto brindado:

In [ ]:
import subprocess
import re

def run_dig(domain):
    result = subprocess.run(['dig', '+short', domain], stdout=subprocess.PIPE)
    return result.stdout.decode('utf-8').strip()

def run_traceroute(ip_address):
    result = subprocess.run(['traceroute', ip_address], stdout=subprocess.PIPE)
    return result.stdout.decode('utf-8')

def get_attack_source(trace_result):
    match = re.search(r'\((\d+\.\d+\.\d+\.\d+)\)', trace_result)
    if match:
        return match.group(1)
    else:
        return None

def block_ip(ip_address):
    cmd = f"iptables -A INPUT -s {ip_address} -j DROP"
    subprocess.run(cmd, shell=True)
    print(f"Blocked IP address {ip_address}")

def analyze_traffic(ip_address):
    # Simulación de análisis de tráfico sospechoso
    if "192.168.1.100" in ip_address:
        return True
    else:
        return False

def main():
    domain = 'suspicious.example.com'

    # Paso 1: Identificar la fuente del ataque utilizando dig y traceroute
    ip_address = run_dig(domain)
    if ip_address:
        print(f"IP Address for {domain} is {ip_address}")
        trace_result = run_traceroute(ip_address)
        print("Traceroute result:", trace_result)
        attack_source = get_attack_source(trace_result)

        # Paso 2: Analizar el tráfico sospechoso
        if attack_source:
            print("Potential attack source:", attack_source)
            if analyze_traffic(attack_source):
                # Paso 3: Implementar estrategias para mitigar y prevenir futuros ataques
                block_ip(attack_source)
                print("Action: Blocked suspicious traffic from potential attack source.")
            else:
                print("Action: No suspicious traffic detected.")
        else:
            print("No attack source found in traceroute result")
    else:
        print(f"Could not resolve domain {domain}")

if __name__ == "__main__":
    main()

Could not resolve domain suspicious.example.com


# **Problema 2: Análisis forense de un ataque de denegación de servicio (DoS)**

**Contexto**: Un servicio en línea de la empresa ha sido interrumpido debido a un ataque DoS.
Se requiere que los estudiantes realicen un análisis forense utilizando registros de eventos y
herramientas de captura de paquetes para identificar la fuente del ataque y proponer
medidas correctivas.

## **Paso 1: Diseño del protocolo**


1. Formato del mensaje:

  * Encabezado:
    - Tipo de operación: Indica la acción que se debe realizar, como PUT, GET o DELETE.

    - Tamaño del mensaje: Especifica la longitud del mensaje en bytes.

    - Número de secuencia: Proporciona un identificador único para cada mensaje enviado en una secuencia.

    - Número de acuse de recibo: Indica el número de secuencia del último mensaje recibido correctamente por el receptor.

  * Cuerpo del mensaje: Contiene los datos asociados con la operación, como el recurso a agregar, obtener o eliminar.

2. Tipos de operaciones:

  * PUT: Utilizado para agregar o actualizar un recurso en el servidor.
  * GET: Utilizado para solicitar un recurso al servidor.
  * DELETE: Utilizado para eliminar un recurso del servidor.

3. Cabecera de control de flujo y manejo de errores:

  * Se debe agregar cabeceras adicionales para el control de flujo y el manejo de errores según sea necesario. Por ejemplo, se podría incluir campos para ventanas de recepción, números de secuencia duplicados o fuera de orden, y códigos de error para manejar situaciones de error.

## **Paso 2: Implementación de control de flujo**

El siguiente código son modificaciones y mejoras realizadas al protocolo propuesto en base al código brindado en la actividad.

In [ ]:
import socket
import struct
# Definición de constantes para tipos de operaciones
PUT = 1
GET = 2
DELETE = 3

# Definición del tamaño de la ventana deslizante
WINDOW_SIZE = 5

# Lista para almacenar los mensajes pendientes de confirmación
pending_messages = []

def send_message(sock, msg_type, seq_num, ack_num, payload):
  header = struct.pack('!B I I I', msg_type, seq_num, ack_num, len(payload))
  message = header + payload.encode()
  sock.sendall(message)

def receive_message(sock):
  header = sock.recv(13) # Tamaño de la cabecera
  msg_type, seq_num, ack_num, length = struct.unpack('!B I I I', header)
  payload = sock.recv(length).decode()
  return msg_type, seq_num, ack_num, payload

def simulate_tcp_connection(host, port):
  server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
  server_sock.bind((host, port))
  server_sock.listen(1)
  client_sock, addr = server_sock.accept()

  # Simulación del envío y recepción de mensajes
  send_message(client_sock, 1, 100, 0, "Hello, this is a test payload")
  print("Message sent")
  msg_type, seq_num, ack_num, payload = receive_message(client_sock)
  if msg_type == PUT:  # Manejar la respuesta PUT
        print("PUT Message received:", payload)
  client_sock.close()
  server_sock.close()


simulate_tcp_connection('localhost', 12345)

## **Paso 3: Evaluación y Optimización**

Ahora vamos a evaluar el rendimeinto usando un enfoque de prueba sistemática para diferentes condiciones de red, incluyendo alta latencia y alta pérdida de paquetes (se ha realizado algunos cambios a funciones para un correcto funcionamiento):

In [ ]:
import socket
import struct
import time
import random

# Definir constantes para tipos de operaciones
PUT = 1
GET = 2
DELETE = 3

# Definir el tamaño de la ventana deslizante
WINDOW_SIZE = 5

# Lista para almacenar los mensajes pendientes de confirmación
pending_messages = []

def send_message(sock, msg_type, seq_num, ack_num, payload):
    header = struct.pack('!B I I I', msg_type, seq_num, ack_num, len(payload))
    message = header + payload.encode()
    sock.sendall(message)

def receive_message(sock):
    header = sock.recv(13) # Tamaño de la cabecera
    msg_type, seq_num, ack_num, length = struct.unpack('!B I I I', header)
    payload = sock.recv(length).decode()
    return msg_type, seq_num, ack_num, payload

def simulate_tcp_connection_with_network_conditions(host, port, delay_prob=0.1, loss_prob=0.1):
    client_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_sock.connect((host, port))
    send_message(client_sock, PUT, 100, 0, "Hello, this is a test payload")
    print("Message sent")

    if random.random() < delay_prob:
        print("Simulating delay...")
        time.sleep(5) # Simular un retraso de 5 segundos

    # Simular pérdida de paquetes
    if random.random() < loss_prob:
        print("Packet loss simulated.")
        return

    # Recibir respuesta y procesarla
    msg_type, seq_num, ack_num, payload = receive_message(client_sock)
    if msg_type == PUT:  # Manejar la respuesta PUT
        print("Received message:", payload)
    client_sock.close()

def simulate_network_conditions(host, port, delay_prob=0.1, loss_prob=0.1):
    server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_sock.bind((host, port))
    server_sock.listen(1)
    client_sock, addr = server_sock.accept()

    # Simulación del envío y recepción de mensajes con condiciones de red
    send_message(client_sock, PUT, 100, 0, "Hello, this is a test payload")
    print("Message sent")
    if random.random() < delay_prob:
        print("Simulating delay...")
        time.sleep(5) # Simular un retraso de 5 segundos

    # Simular pérdida de paquetes
    if random.random() < loss_prob:
        print("Packet loss simulated.")
        return

    msg_type, seq_num, ack_num, payload = receive_message(client_sock)
    if msg_type == PUT:  # Manejar la respuesta PUT
        print("Received message:", payload)

    client_sock.close()
    server_sock.close()

# Ejemplo de uso
simulate_network_conditions('localhost', 12345, delay_prob=0.2, loss_prob=0.1)

# **Problema 3: Implementación y análisis de una red segura utilizando varias herramientas**

**Contexto:** Diseña una red para una pequeña empresa que incluya servidores web, de correo
electrónico y bases de datos. Utiliza ifconfig o ipconfig para configurar las interfaces de red,
dig y whois para configurar los registros DNS, y cURL o wget para probar la accesibilidad de
los servicios. Además, implementa estrategias de defensa en profundidad como zonas
desmilitarizadas (DMZ) y subredes internas.

## **1. Configuración de la red**
Supongamos que estamos configurando una red que incluye un servidor web, un servidor
de correo electrónico y un servidor de base de datos. Utilizaremos ifconfig o el moderno ip
para configurar las interfaces de red.
Configuración de IP estática en Linux.

In [ ]:
# Configurar una dirección IP estática en la interfaz eth0

!sudo ip addr add 192.168.1.100/24 dev eth0 # añade una dirección IP a la interfaz especificada.
!sudo ip link set eth0 up # activamos la interfaz de red.

## **2. Configuración de DNS con dig y whois**
Para configurar y verificar registros DNS, así como para obtener información de dominio:

In [ ]:
# Obtener registros A de un dominio
!dig +short example.com A
# Obtener información del dominio
!whois example.com

## **3. Scripting para automatización y monitoreo**

Este script es una versión mejorada de la que se brindo en clase:

* Se agregó manejo de excepciones para tratar posibles errores durante la ejecución del script.
* La función check_server ahora ejecuta traceroute en caso de fallo de ping y envía un correo electrónico con los resultados.
* Se agregó un bucle infinito que verifica periódicamente la disponibilidad de los servidores cada minuto.

In [ ]:
import subprocess
import smtplib
from email.mime.text import MIMEText
import time

def send_email(message):
    sender = "monitor@example.com"
    receivers = ["admin@example.com"]
    msg = MIMEText(message)
    msg['Subject'] = "Alerta de Red"
    msg['From'] = sender
    msg['To'] = ", ".join(receivers)
    try:
        with smtplib.SMTP('localhost') as server:
            server.sendmail(sender, receivers, msg.as_string())
        print("Alerta enviada por email")
    except smtplib.SMTPException as e:
        print(f"Error al enviar correo electrónico: {e}")

def check_server(address):
    try:
        result = subprocess.run(['ping', '-c', '3', address], stdout=subprocess.PIPE, timeout=10)
        if result.returncode != 0:
            print(f"Fallo en la conexión con {address}")
            traceroute_result = subprocess.run(['traceroute', address], stdout=subprocess.PIPE, timeout=10)
            send_email(f"Fallo en la conexión con {address}\n\nTraceroute:\n{traceroute_result.stdout.decode('utf-8')}")
    except subprocess.TimeoutExpired:
        print(f"Tiempo de espera agotado para {address}")
        send_email(f"Tiempo de espera agotado para {address}")

if __name__ == "__main__":
    servers = ["192.168.1.100", "192.168.1.101", "192.168.1.102"]
    while True:
        for server in servers:
            check_server(server)
        # Espera 1 minuto antes de volver a verificar
        time.sleep(60)


## **4. Uso de cURL o wget para pruebas**

En el siguiente código:

* Se utiliza cURL para verificar la accesibilidad de los servidores web.
* El script verifica el código de estado HTTP devuelto por el servidor web.
* Si el código de estado es 200 (OK), el servidor web se considera accesible.
* Si el código de estado no es 200, se envía una alerta por correo electrónico.
* El script se ejecuta en un bucle infinito, verificando periódicamente la accesibilidad de los servidores web.

In [ ]:
import subprocess
import smtplib
from email.mime.text import MIMEText
import time

def send_email(message):
    sender = "monitor@example.com"
    receivers = ["admin@example.com"]
    msg = MIMEText(message)
    msg['Subject'] = "Alerta de Red"
    msg['From'] = sender
    msg['To'] = ", ".join(receivers)
    try:
        with smtplib.SMTP('localhost') as server:
            server.sendmail(sender, receivers, msg.as_string())
        print("Alerta enviada por email")
    except smtplib.SMTPException as e:
        print(f"Error al enviar correo electrónico: {e}")

def check_server(address):
    try:
        # Verificar la accesibilidad del servidor web con cURL
        curl_output = subprocess.check_output(['curl', '-o', '/dev/null', '-s', '-w', '%{http_code}', address])
        http_status_code = int(curl_output.decode('utf-8'))
        if http_status_code == 200:
            print(f"Servidor web accesible: {address}")
        else:
            print(f"Error: HTTP status code {http_status_code} al acceder a {address}")
            send_email(f"Error: HTTP status code {http_status_code} al acceder a {address}")
    except subprocess.CalledProcessError as e:
        print(f"Error al ejecutar cURL: {e}")
        send_email(f"Error al ejecutar cURL: {e}")

if __name__ == "__main__":
    servers = ["http://192.168.1.100", "http://192.168.1.101", "http://192.168.1.102"]
    while True:
        for server in servers:
            check_server(server)
        # Espera 1 minuto antes de volver a verificar
        time.sleep(60)

## **5. Implementar seguridad**
Podríamos configurar firewalls utilizando iptables para limitar el acceso no autorizado a los
servicios:

In [ ]:
# Bloquear todo el tráfico excepto el que viene de las IPs autorizadas
!sudo iptables -A INPUT -p tcp -s 192.168.1.0/24 -j ACCEPT
!sudo iptables -A INPUT -p tcp -j DROP

# **Problema 4: Desarrollo de un script para monitoreo y alerta de red**

**Problema**: Escribe un script utilizando bash o python que utilice herramientas como ping,
traceroute, y nslookup para monitorear la disponibilidad de varios servidores críticos. El
script debe registrar cualquier anomalía y enviar alertas automáticas si detecta problemas
como tiempos de respuesta elevados o pérdida de paquetes.

En este código:

* Las credenciales del servidor SMTP se obtienen de variables de entorno para mantenerlas seguras.
* Se implementa un bucle de reintento en caso de que ocurran errores temporales, con un retraso entre intentos.
* El código está listo para expandirse añadiendo más comprobaciones, como verificar puertos específicos con nmap o comprobar la carga de los servidores con ssh y comandos como top.

In [ ]:
import subprocess
import smtplib
from email.mime.text import MIMEText
import datetime
import os
import time

def send_email(subject, message):
    sender = os.getenv('SENDER_EMAIL')
    receivers = os.getenv('RECEIVER_EMAIL').split(',')
    smtp_server = os.getenv('SMTP_SERVER')
    smtp_port = os.getenv('SMTP_PORT')
    smtp_username = os.getenv('SMTP_USERNAME')
    smtp_password = os.getenv('SMTP_PASSWORD')

    msg = MIMEText(message)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ", ".join(receivers)

    # Configurar el servidor SMTP según la configuración segura
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)
    server.sendmail(sender, receivers, msg.as_string())
    server.quit()
    print("Alerta enviada por correo electrónico")

def check_server(address):
    response = subprocess.run(['ping', '-c', '3', address], stdout=subprocess.PIPE, text=True)
    if response.returncode != 0:
        now = datetime.datetime.now()
        message = f"Fallo de ping a {address} el {now.strftime('%Y-%m-%d %H:%M:%S')}"
        send_email("Alerta de Fallo de Red", message)
        log_event(message)
    else:
        print(f"{address} está activo")

def trace_route(address):
    result = subprocess.run(['traceroute', address], stdout=subprocess.PIPE, text=True)
    log_event(f"Traceroute a {address}:\n{result.stdout}")

def log_event(message):
    with open("network_monitor_log.txt", "a") as file:
        file.write(message + "\n")

if __name__ == "__main__":
    servers = ["192.168.1.100", "192.168.1.101", "192.168.1.102"]
    retry_attempts = 3
    retry_delay = 10

    while True:
        for server in servers:
            for attempt in range(retry_attempts):
                try:
                    check_server(server)
                    trace_route(server)
                    break  # Éxito, salir del bucle de reintento
                except Exception as e:
                    print(f"Error: {e}. Reintentando en {retry_delay} segundos...")
                    time.sleep(retry_delay)
                    continue
        time.sleep(300)  # Esperar 5 minutos antes de volver a verificar